# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [43]:
# Dependencies and Setup
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [44]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/flynnlives/Documents/GitHub/UCI_DABC_Python_API_Challenge/WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bayganin,48.6947,55.8742,87.35,26,56,9.62,KZ,1691252539
1,1,belmonte,-15.8631,-38.8828,78.55,51,0,6.53,BR,1691252582
2,2,magadan,59.5667,150.8000,53.22,62,100,7.56,RU,1691252582
3,3,port elizabeth,-33.9180,25.5701,72.10,82,0,0.00,ZA,1691252582
4,4,edinburgh of the seven seas,-37.0676,-12.3116,55.81,79,8,31.36,SH,1691252582


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
gmaps.configure(geoapify_key)
humidity = city_data_df["Humidity"].astype(float)
locations = city_data_df[["Lat", "Lng"]].astype(float)
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

figure = gmaps.figure(map_type="HYBRID", layout=layout)
layer = gmaps.heatmap_layer(locations, weights=humidity,
                    dissipating=False, max_intensity=100,
                    point_radius=2)
figure.add_layer(layer)

# Display the map
figure

Figure(layout=FigureLayout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px s…

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [62]:
# Narrow down cities that fit criteria and drop any results with null values
pref_weather_df = pd.DataFrame(city_data_df, columns=['City', 'Max Temp', 'Wind Speed', 'Cloudiness', 'Country', 'Lat', 'Lng'])

max_temp = (pref_weather_df['Max Temp'] <= 80) & (pref_weather_df['Max Temp'] >= 65)
wind_speed = (pref_weather_df['Wind Speed'] < 10)
cloudiness = (pref_weather_df['Cloudiness'] == 0)

# Drop any rows with null values

# Display sample data
pref_weather_df[max_temp & wind_speed & cloudiness]


,City,Max Temp,Wind Speed,Cloudiness,Country,Lat,Lng
1,belmonte,78.55,6.53,0,BR,-15.8631,-38.8828
3,port elizabeth,72.10,0.00,0,ZA,-33.9180,25.5701
30,rakops,65.91,6.67,0,BW,-21.0167,24.3333
53,hawaiian paradise park,71.04,6.91,0,US,19.5933,-154.9731
62,port alfred,66.18,9.04,0,ZA,-33.5906,26.8910
111,minot,75.31,9.22,0,US,48.2325,-101.2963
113,labytnangi,71.69,7.47,0,RU,66.6572,66.4183
131,vorgashor,68.92,8.08,0,RU,67.5833,63.9500
141,kendari,74.19,2.62,0,ID,-3.9450,122.4989
146,rio grande,66.31,7.81,0,BR,-32.0350,-52.0986


### Step 3: Create a new DataFrame called `hotel_df`.

In [63]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(pref_weather_df, columns=['City', 'Country', 'Lat', 'Lng'])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,bayganin,KZ,48.6947,55.8742,
1,belmonte,BR,-15.8631,-38.8828,
2,magadan,RU,59.5667,150.8000,
3,port elizabeth,ZA,-33.9180,25.5701,
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,
...,...,...,...,...,...
568,boundiali,CI,9.5217,-6.4869,
569,menaka,ML,15.9182,2.4022,
570,bitam,GA,2.0760,11.5007,
571,villa gesell,AR,-37.2639,-56.9730,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [64]:
# Set parameters to search for a hotel

coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
search = 'hotel'
radius = 10000
tar_type = 'hotel'

params = {
    "location": coordinates,
    "radius": radius,
    "type": tar_type,
    "keyword": search,
    "key": geoapify_key,
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# assemble url and make API request
response = requests.get(base_url, params=params).json()
results = response['results']

hotel_df.iloc[0][0]

'bayganin'

In [65]:
# Print a message to follow up the hotel search
print("Starting hotel search")

hotel_name = []

params = {
    "radius": 10000,
    "type": 'hotel',
    "keyword": 'hotel',
    "key": geoapify_key
}

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address = requests.get(base_url, params=params)
    address = address.json()
            
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_name.append(address['results'][0]['name'])
        
    except IndexError:
        hotel_name.append(np.nan)
        print(f"Missing field/result for {city}... Not Found.")
        
    print("------------")

Starting hotel search
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo... Not Found.
------------
Missing field/result for huambo...

In [66]:
 # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df['Hotel Name'] = hotel_name
hotel_df.dropna()
hotel_df

betio village - nearest hotel: 


,City,Country,Lat,Lng,Hotel Name
0,bayganin,KZ,48.6947,55.8742,NaN
1,belmonte,BR,-15.8631,-38.8828,NaN
2,magadan,RU,59.5667,150.8000,NaN
3,port elizabeth,ZA,-33.9180,25.5701,NaN
4,edinburgh of the seven seas,SH,-37.0676,-12.3116,NaN
...,...,...,...,...,...
568,boundiali,CI,9.5217,-6.4869,NaN
569,menaka,ML,15.9182,2.4022,NaN
570,bitam,GA,2.0760,11.5007,NaN
571,villa gesell,AR,-37.2639,-56.9730,NaN


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
%%capture --no-display

# Configure the map plot
details_template = '''
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
'''
hotel_details = [details_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']]

markers = gmaps.marker_layer(locations)
figure.add_layer(markers)

# Display the map
figure

AttributeError: module 'collections' has no attribute 'Iterable'